In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import tensorflow as tf


## Reading the data

In [3]:
data = pd.read_csv(r"..\..\Data\small_ohe.csv")
data.head()

,age,default,housing,loan,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,...,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed
0,0.171429,1,-1,1,0.029412,1.0,0.0,0,0.333333,0.269680,...,0,1,0,0,0,1,0,0,0,0
1,0.300000,1,1,1,0.088235,1.0,0.0,0,0.937500,0.698753,...,0,1,0,0,0,1,0,0,0,0
2,0.100000,1,-1,1,0.000000,1.0,0.0,0,1.000000,0.882307,...,0,0,0,0,0,0,0,0,0,1
3,0.285714,1,0,0,0.058824,1.0,0.0,0,1.000000,0.882307,...,0,0,0,0,0,1,0,0,0,0
4,0.414286,1,-1,1,0.000000,1.0,0.0,0,0.687500,0.389322,...,0,0,1,0,0,0,1,0,0,0


In [4]:
data.columns

Index(['age', 'default', 'housing', 'loan', 'campaign', 'pdays', 'previous',
       'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
       'euribor3m', 'nr.employed', 'y', 'pdays2', 'job_admin.',
       'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'contact_cellular', 'contact_telephone',
       'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
       'day_of_week_fri', 'day_of_week_mon', 'day_of_week_thu',
       'day_of_w

In [5]:
# Saperating features and result vectors
X = data.drop('y', axis=1).values
y = data['y'].values

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
X_train, X_calibration, y_train, y_calibration = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

In [7]:
# Define no sequence of layers
input_tensor = tf.keras.Input(shape=(55,))
layer1 = tf.keras.layers.Dense(64,name = 'layer_1', activation='relu',use_bias=True)(input_tensor)
layer2 = tf.keras.layers.Dense(64,name = 'layer_2', activation='relu',use_bias=True)(layer1)
layer3 = tf.keras.layers.Dense(32,name = 'layer_3', activation='relu',use_bias=True)(layer2)
layer4 = tf.keras.layers.Dense(1, name = 'logit', activation=None,use_bias=True)(layer3)
output_tensor = tf.keras.layers.Activation('sigmoid', name='output')(layer4)
model = tf.keras.Model(inputs=input_tensor, outputs=[layer1,layer2,layer3,layer4,output_tensor]) 
losses = {'logit': 'BinaryCrossentropy'}
model.compile(loss=losses, optimizer='adam', metrics=['accuracy'])

In [8]:
model.fit(X_train, y_train, batch_size=128, epochs=200, validation_data=(X_test, y_test))

Epoch 1/200
41/41 [==============================] - 1s 5ms/step - loss: 0.7808 - logit_loss: 0.7808 - layer_1_accuracy: 0.0624 - layer_2_accuracy: 1.9091e-04 - layer_3_accuracy: 0.0867 - logit_accuracy: 0.5670 - output_accuracy: 0.4975 - val_loss: 0.5828 - val_logit_loss: 0.5828 - val_layer_1_accuracy: 0.0481 - val_layer_2_accuracy: 0.0027 - val_layer_3_accuracy: 0.0783 - val_logit_accuracy: 0.7239 - val_output_accuracy: 0.4973
Epoch 2/200
41/41 [==============================] - 0s 2ms/step - loss: 0.5974 - logit_loss: 0.5974 - layer_1_accuracy: 0.0550 - layer_2_accuracy: 0.0011 - layer_3_accuracy: 0.0825 - logit_accuracy: 0.7167 - output_accuracy: 0.4987 - val_loss: 0.5357 - val_logit_loss: 0.5357 - val_layer_1_accuracy: 0.0440 - val_layer_2_accuracy: 0.0027 - val_layer_3_accuracy: 0.0618 - val_logit_accuracy: 0.7624 - val_output_accuracy: 0.4973
Epoch 3/200
41/41 [==============================] - 0s 1ms/step - loss: 0.5504 - logit_loss: 0.5504 - layer_1_accuracy: 0.0554 - layer_2_

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 55)]              0         
                                                                 
 layer_1 (Dense)             (None, 64)                3584      
                                                                 
 layer_2 (Dense)             (None, 64)                4160      
                                                                 
 layer_3 (Dense)             (None, 32)                2080      
                                                                 
 logit (Dense)               (None, 1)                 33        
                                                                 
 output (Activation)         (None, 1)                 0         
                                                                 
Total params: 9,857
Trainable params: 9,857
Non-trainable par

## Convert NN to ONNX file

In [10]:
import onnxmltools
# convert the model to ONNX format
onnx_net = onnxmltools.convert_keras(model)
onnxmltools.utils.save_model(onnx_net, "my_model.onnx")
content = onnx_net.SerializeToString()

## Setup onnx

In [11]:
import onnxruntime as ort
from sklearn.neighbors import NearestNeighbors
def setup(onnx_file: str,):
    # Load the ONNX model
    ort_sess = ort.InferenceSession(onnx_file)
    ortvalue = ort.OrtValue.ortvalue_from_numpy(X_train.astype(np.float32), 'cpu')
    layer_output = ort_sess.run(['layer_1','layer_2','layer_3','logit'], {ort_sess.get_inputs()[0].name: ortvalue}) 
    neigh = []
    for i in range(len(layer_output)):
        neigh.append(NearestNeighbors().fit(layer_output[i]))
    return  ort_sess, neigh

ort_sess, neigh = setup('my_model.onnx')

In [13]:
def confidences(x, k, ort_sess, neigh, y_train, x_calibration, y_calibration):
    ortvalue = ort.OrtValue.ortvalue_from_numpy(x.astype(np.float32), 'cpu')

    # computer  layer_wise kNN for each sample
    layer_output = ort_sess.run(['layer_1','layer_2','layer_3','logit'], {ort_sess.get_inputs()[0].name: ortvalue})
    omega = []
    for j in range(len(layer_output)):
        index = neigh[j].kneighbors(layer_output[j], k, return_distance=False)
        c = y_train[index]
        omega.append(c)

    # compute test input alpha
    omega = np.array(omega)
    omega = np.transpose(omega, (1, 0, 2))
    alpha_0 = np.sum(omega, axis=2)
    alpha_0 = np.sum(alpha_0, axis=1)
    alpha_1 = len(layer_output) * k - alpha_0

    # compute calibrations alpha
    ortvalue = ort.OrtValue.ortvalue_from_numpy(x_calibration.astype(np.float32), 'cpu')
    layer_output = ort_sess.run(['layer_1','layer_2','layer_3','logit'], {ort_sess.get_inputs()[0].name: ortvalue})
    A_omega = []
    for j in range(len(layer_output)):
        index = neigh[j].kneighbors(layer_output[j], 5, return_distance=False)
        c = y_train[index]
        A_omega.append(c)

    A_omega = np.array(A_omega)
    A_omega = np.transpose(A_omega, (1, 0, 2))
    c_alpha_0 = np.sum(A_omega, axis=2)
    c_alpha_0 = np.sum(c_alpha_0, axis=1)
    c_alpha_1 = len(layer_output) * k - c_alpha_0
    alpha = np.zeros(len(y_calibration))
    alpha[y_calibration==1] = c_alpha_1[y_calibration==1]

    # compute confidence
    p_0 = [sum(alpha > i)/len(alpha) for i in alpha_0]
    p_1 = [sum(alpha > i)/len(alpha) for i in alpha_1]
    p_0 = np.array(p_0)
    p_1 = np.array(p_1)
    p = np.array([p_0, p_1])
    prediction = np.argmax([p_0, p_1], axis=0)
    confidence = np.zeros(len(prediction))
    confidence[prediction == 0] = 1 - p_0[prediction == 0]
    confidence[prediction == 1] = 1 - p_1[prediction == 1]

    return prediction, confidence

k = 5
prediction, confidence = confidences(X_test, k, ort_sess, neigh, y_train, X_calibration, y_calibration)
print("prediction: ", prediction) 
print("confidence: ", confidence)

prediction:  [1 1 1 0 1 0 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 0 0 0 1 0
 0 0 0 0 0 0 1 0 1 0 1 0 1 1 1 0 1 0 1 1 0 0 1 1 0 1 1 1 1 1 1 0 1 0 0 1 1
 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 1 1 1 0 1 1 1 1 1 1 0 0 1 0 0 0
 0 0 0 0 0 1 1 1 0 0 0 0 1 1 0 0 1 1 1 1 1 0 1 1 0 0 0 0 1 0 1 1 0 1 0 1 1
 1 1 1 0 0 1 1 0 0 1 0 0 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 0
 1 1 0 0 0 1 1 1 0 1 0 1 1 0 0 0 0 1 0 1 1 0 1 1 0 0 1 1 1 0 1 0 0 1 1 0 1
 0 0 0 0 0 1 1 1 1 0 1 0 1 1 1 0 1 0 1 1 1 0 0 1 1 0 1 1 1 0 1 0 0 0 0 1 1
 1 1 0 0 0 1 0 1 1 0 0 1 1 1 1 0 1 1 0 1 0 1 0 1 0 0 1 0 0 1 1 0 0 0 1 0 1
 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 1 0 1 1 1 0 0 1 0 1 1 0 1 1 0 0
 1 0 1 1 1 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 1 0 1 1 1 1 1
 0 0 1 0 0 0 0 1 1 0 1 1 1 0 1 1 1 0 1 0 1 0 1 1 0 1 1 0 1 1 0 0 1 1 0 0 1
 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 0 1 1 0 1 1 0 0 1 0 0 1 1 1
 1 0 0 1 0 0 1 1 1 1 0 1 0 1 1 0 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 0
 1 1 1 1 0 0